In [ ]:
import os
import random
import numpy as np
import pickle

import matplotlib.pyplot as plt
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

import utils
from base import Base

In [ ]:
class Assignment2(Base):

    def __init__(self):
        super(Assignment2, self).__init__()
        self.data = pickle.load(open('./features/cifar10/raw.pkl', 'rb'))
        self.kmeans = self.get_model()
        self.closest = None

    def get_model(self):
        """
        TO BE IMPLEMENTED BY STUDENT

        """
        pass

    def train(self, train=True):
        model_file = 'models/kmeans.pkl'
        if train:
            data = self.data.reshape(len(self.data), -1)
            self.kmeans.fit(data)
            self.closest, _ = pairwise_distances_argmin_min(self.kmeans.cluster_centers_, data)
            with open(model_file, 'wb') as f:
                pickle.dump((self.kmeans, self.closest), f)
        else:
            with open(model_file, 'rb') as f:
                self.kmeans, self.closest = pickle.load(f)

    def get_patch(self, tile):
        cluster_indices = self.kmeans.predict(tile.reshape(1, -1))
        patch = self.data[self.closest[cluster_indices[0]]]
        return patch

In [ ]:
os.makedirs('output/A2/mosaics/', exist_ok=True)
os.makedirs('models', exist_ok=True)

In [ ]:
"""
Assignment 2 - Clustering

"""

# The program will start execution here
# Change the filename to load your favourite picture
file = './images/eye2.jpg'
train = False

img = Image.open(file).convert('RGB')
img = utils.resize_proportional(img, new_height=900)
target_image = np.array(img) / 255

# This will execute the Mosaicking algorithm of Assignment 2
main = Assignment2()
main.train(train)
output_image = main.mosaic(target_image)

# Saving the image inside in project root folder
output_image *= 255
im = Image.fromarray(output_image.astype('uint8'))
im.save(utils.datetime_filename('output/A2/mosaics/mosaic.png'))

# Testing

In [ ]:
# Setup
main = Assignment2()
main.train(False)
make_folders()
    
os.makedirs('output/A2_test/clusters', exist_ok=True)

In [ ]:
# test cluster
clusters = [5, 6, 7]
num_cols = 5
num_rows = len(clusters)

i = 0
for row in range(num_rows):
    cluster = clusters[row]
    cluster_center = main.data[main.closest[cluster]]
    labels = main.kmeans.labels_
    cluster_indices = [i for i, l in enumerate(labels) if l == cluster]

    d = main.kmeans.transform(main.data.reshape(len(main.data), -1))[:, cluster]
    ind = np.argsort(d)[:num_cols]
    closest = main.data[ind]

    for col in range(num_cols):
        i += 1

        # else:
        # patch_idx = random.choice(cluster_indices)
        # patch = main.data[patch_idx]
        patch = closest[col]
        # grid plot of size 2 x num_cols
        plt.subplot(num_rows, num_cols, i)
        if col == 0:
            plt.title('Center')
        # plt.title(str(patch_idx))
        plt.imshow(patch)

fig = plt.gcf()
plt.show()
fname = utils.datetime_filename('output/A2_test/clusters/samples.png')
fig.savefig(fname, format='png', dpi=300)